# Table of Contents

0. [Import Libraries](#0-import-libraries)  
1. [Import Data](#1-import-data)  
2. [Data Exploration](#2-data-exploration)  
3. [Data Cleaning](#3-data-cleaning)  
4. [Data Preparation](#4-data-preparation)  
5. [Benchmark Model](#5-benchmark-model)  
6. [Feature Engineering](#6-feature-engineering)  
7. [Model Training](#7-model-training)  
    7.1 [Decision Tree](#71-decision-tree)  
    7.2 [Random Forest](#72-random-forest)  
    7.3 [Comparing Algorithm Performances](#73-comparing-algorithm-performances)  
8. [Tuning](#8-tuning)  
9. [Conclusion](#9-conclusion)  



## **Data Overview**

### **0. Import Libraries**
[🔝 Back to Top](#table-of-contents)


In [78]:
import pandas as pd
import numpy as np

In [79]:
df = pd.read_csv('../data/raw/train/df_Payments.csv')

In [80]:
df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,Axfy13Hk4PIk,1,credit_card,1,259.14
1,v6px92oS8cLG,1,credit_card,8,382.39
2,Ulpf9skrhjfm,1,credit_card,4,249.25
3,bwJVWupf2keN,1,credit_card,2,27.79
4,Dd0QnrMk9Cj5,1,credit_card,1,76.15


In [81]:
print(df.shape)

(89316, 5)


In [82]:
df['payment_installments'].unique()

array([ 1,  8,  4,  2,  5,  6, 10,  3,  7,  9, 18, 12, 24, 15, 11, 13, 21,
       16, 20, 14, 17, 23, 22,  0], dtype=int64)

In [83]:
df[df['payment_sequential'] == 2].head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
24,xfXLgLbCo5BC,2,credit_card,1,166.71
39,0QqNX4xLriht,2,voucher,1,1843.85
102,pHtHYRoip8Cl,2,voucher,1,206.07
103,yGQcdep9edOe,2,voucher,1,145.48
135,gQPaVIZUm0jj,2,voucher,1,163.33


In [84]:
df['payment_type'].value_counts()

payment_type
credit_card    65814
wallet         17302
voucher         4911
debit_card      1289
Name: count, dtype: int64

In [85]:
products_data = pd.read_csv('../data/raw/train/df_Products.csv')

In [86]:
products_data.head()

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,90K0C1fIyQUf,toys,491.0,19.0,12.0,16.0
1,qejhpMGGVcsl,watches_gifts,440.0,18.0,14.0,17.0
2,qUS5d2pEAyxJ,costruction_tools_garden,2200.0,16.0,16.0,16.0
3,639iGvMyv0De,toys,1450.0,68.0,3.0,48.0
4,1lycYGcsic2F,toys,300.0,17.0,4.0,12.0


In [87]:
products_data['product_category_name'].value_counts()

product_category_name
toys                                 67027
health_beauty                         2351
bed_bath_table                        2146
sports_leisure                        1837
furniture_decor                       1760
                                     ...  
fashion_childrens_clothes                2
security_and_services                    1
furniture_mattress_and_upholstery        1
home_comfort_2                           1
diapers_and_hygiene                      1
Name: count, Length: 70, dtype: int64

## 9. Conclusion
[Back To Top](#table-of-contents)